## Deliverable 2. Create a Customer Travel Destinations Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Alice Springs,AU,-23.7000,133.8833,93.99,36,74,13.80,broken clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,81.99,76,35,12.93,moderate rain
2,2,Leh,IN,34.1667,77.5833,3.54,66,0,1.59,clear sky
3,3,Iqaluit,CA,63.7506,-68.5145,-13.00,70,1,27.63,snow
4,4,Krasnyy Chikoy,RU,50.3643,108.7512,8.82,90,100,2.84,overcast clouds
5,5,Cape Town,ZA,-33.9258,18.4232,70.00,94,75,3.44,broken clouds
6,6,Pisco,PE,-13.7000,-76.2167,71.60,73,0,6.91,clear sky
7,7,San Cristobal,VE,7.7669,-72.2250,63.48,76,35,1.54,scattered clouds
8,8,Arraial Do Cabo,BR,-22.9661,-42.0278,76.71,84,38,24.09,scattered clouds
9,9,Erenhot,CN,43.6475,111.9767,20.84,90,0,20.09,clear sky


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hithadhoo,MV,-0.6000,73.0833,81.99,76,35,12.93,moderate rain
5,5,Cape Town,ZA,-33.9258,18.4232,70.00,94,75,3.44,broken clouds
6,6,Pisco,PE,-13.7000,-76.2167,71.60,73,0,6.91,clear sky
8,8,Arraial Do Cabo,BR,-22.9661,-42.0278,76.71,84,38,24.09,scattered clouds
10,10,Mayumba,GA,-3.4320,10.6554,78.28,89,100,7.02,light rain
13,13,Puerto Ayora,EC,-0.7393,-90.3518,77.00,94,38,5.01,light rain
21,21,Busselton,AU,-33.6500,115.3333,84.00,41,0,5.99,clear sky
22,22,Hilo,US,19.7297,-155.0900,75.20,73,90,6.91,moderate rain
23,23,Vaini,TO,-21.2000,-175.2000,80.60,89,90,14.97,light rain
24,24,Hobart,AU,-42.8794,147.3294,75.20,50,75,11.50,broken clouds


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                213
City                   213
Country                213
Lat                    213
Lng                    213
Max Temp               213
Humidity               213
Cloudiness             213
Wind Speed             213
Current Description    213
dtype: int64

In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                213
City                   213
Country                213
Lat                    213
Lng                    213
Max Temp               213
Humidity               213
Cloudiness             213
Wind Speed             213
Current Description    213
dtype: int64

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,81.99,moderate rain,-0.6000,73.0833,
5,Cape Town,ZA,70.00,broken clouds,-33.9258,18.4232,
6,Pisco,PE,71.60,clear sky,-13.7000,-76.2167,
8,Arraial Do Cabo,BR,76.71,scattered clouds,-22.9661,-42.0278,
10,Mayumba,GA,78.28,light rain,-3.4320,10.6554,
13,Puerto Ayora,EC,77.00,light rain,-0.7393,-90.3518,
21,Busselton,AU,84.00,clear sky,-33.6500,115.3333,
22,Hilo,US,75.20,moderate rain,19.7297,-155.0900,
23,Vaini,TO,80.60,light rain,-21.2000,-175.2000,
24,Hobart,AU,75.20,broken clouds,-42.8794,147.3294,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,81.99,moderate rain,-0.6000,73.0833,Scoop Guest House
5,Cape Town,ZA,70.00,broken clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
6,Pisco,PE,71.60,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
8,Arraial Do Cabo,BR,76.71,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
10,Mayumba,GA,78.28,light rain,-3.4320,10.6554,LIKWALE LODGE
13,Puerto Ayora,EC,77.00,light rain,-0.7393,-90.3518,Castillo Galapagos
21,Busselton,AU,84.00,clear sky,-33.6500,115.3333,Observatory Guest House
22,Hilo,US,75.20,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
23,Vaini,TO,80.60,light rain,-21.2000,-175.2000,Keleti Beach Resort
24,Hobart,AU,75.20,broken clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania


In [35]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))